In [1]:
import pandas as pd
import os
import sys
import boto3
import io
import geopandas as gpd

# suppress pandas purely educational warnings
from warnings import simplefilter
simplefilter(action="ignore", category=pd.errors.PerformanceWarning)

sys.path.append(os.path.expanduser('../../'))
from scripts.utils.file_helpers import pull_gpkg_from_directory, upload_csv_aws, filter_counties
from scripts.utils.write_metadata import append_metadata

In [81]:
# pull csv from aws
bucket_name = 'ca-climate-index'
aws_dir = '2b_reproject/built_environment/transportation/cdot/'

pull_gpkg_from_directory(bucket_name, aws_dir)

Saved GeoPackage as 'built_caltrans_airports.gpkg' locally
Saved GeoPackage as 'built_caltrans_bridges.gpkg' locally
Saved GeoPackage as 'built_caltrans_highways.gpkg' locally
Saved GeoPackage as 'built_caltrans_rails.gpkg' locally
Saved GeoPackage as 'built_caltrans_road_bottlenecks.gpkg' locally


In [74]:
airport_data = gpd.read_file('built_caltrans_airports.gpkg')
bridge_data = gpd.read_file('built_caltrans_bridges.gpkg')
'''highway_data = gpd.read_file('built_caltrans_airports.gpkg')
bottleneck_data = gpd.read_file('built_caltrans_airports.gpkg')
freight_rail_data = gpd.read_file('built_caltrans_airports.gpkg')'''

"highway_data = gpd.read_file('built_caltrans_airports.gpkg')\nbottleneck_data = gpd.read_file('built_caltrans_airports.gpkg')\nfreight_rail_data = gpd.read_file('built_caltrans_airports.gpkg')"

In [46]:
airport_data.columns

Index(['OBJECTID', 'FACILITY', 'MAPLABEL', 'AIRPORTID', 'CITY', 'COUNTY',
       'FNCTNLCLSS', 'STATECLASS', 'FAASRVCLVL', 'FAASITENO', 'DISTRICT',
       'MANAGER', 'MNGREMAIL', 'PHONE', 'F5010URL', 'LATDD', 'LONGDD',
       'LATDMS', 'PMTLAT', 'LONGDMS', 'ARPLATDMS', 'ARPLONGDMS', 'PMTLONG',
       'USCB_STATEFP', 'USCB_COUNTYFP', 'USCB_TRACTCE', 'USCB_GEOID',
       'USCB_NAME', 'USCB_NAMELSAD', 'USCB_MTFCC', 'USCB_FUNCSTAT',
       'USCB_ALAND', 'USCB_AWATER', 'USCB_INTPTLAT', 'USCB_INTPTLON',
       'geometry'],
      dtype='object')

In [50]:
bridge_data.

,OBJECTID,DIST,CO,RTE,PM,BRIDGE,BRIDGE_X,BRIDGE_Y,CITY,LAT,LON,NAME,LOC,YRBLT,HST,FAC,APWID,LENG,DK_AREA,LSW,RSW,RDW,REFVCU,VCU,MAINSPANS,DIR,PRINC,INTERSEC,AADT,PCTTRK,DEF,NHS,FUNCTIONAL,DATA_EXTRA,USCB_STATEFP,USCB_COUNTYFP,USCB_TRACTCE,USCB_GEOID,USCB_NAME,USCB_NAMELSAD,USCB_MTFCC,USCB_FUNCSTAT,USCB_ALAND,USCB_AWATER,USCB_INTPTLAT,USCB_INTPTLON,geometry
0,1,1,DN,None,None,01C0001,-0.862156,9.486604e+15,00000,41.523222,-124.032594,Hoppow Creek,500' E Of Rt 101/169 Sep,1965,5,Klamath Mill Road,8.5,17.7,200,1.5,0.6,8.5,N,0.0,2,2,1,Hoppow Creek,1395,25,0,0,9,30-JAN-20,06,015,000203,06015000203,2.03,Census Tract 2.03,G5020,S,368732963,35058418,+41.5767503,-124.0277960,POINT (-124.03260 41.52322)
1,2,1,DN,None,None,01C0002,-54.146810,9.519316e+15,00000,41.523681,-124.035647,Hoppow Creek,West Of Route 101/169 Sep,1965,5,Chapman Street,9.4,31.0,381,0.6,1.5,8.5,N,0.0,1,2,1,Hoppow Creek,20,0,0,0,9,30-JAN-20,06,015,000203,06015000203,2.03,Census Tract 2.03,G5020,S,368732963,35058418,+41.5767503,-124.0277960,POINT (-124.03565 41.52368)
2,7,1,DN,None,None,01C0011,-42.523364,9.819594e+15,00000,41.553397,-124.057572,Hunter Creek,0.1 Mi West Of Rt101,1949,5,Requa Road,8.4,41.6,356,0.0,0.0,8.4,N,0.0,1,2,1,Hunter Creek,1110,2,0,0,8,30-JAN-20,06,015,000203,06015000203,2.03,Census Tract 2.03,G5020,S,368732963,35058418,+41.5767503,-124.0277960,POINT (-124.05757 41.55339)
3,8,1,DN,None,None,01C0012,37.575706,9.853622e+15,00000,41.552528,-124.060939,Salt Creek,0.4 Mi West Of Rte 101,1958,5,Requa Road,8.2,7.0,73,0.0,0.0,0.0,N,0.0,2,2,1,Salt Creek,1110,2,0,0,8,30-JAN-20,06,015,000203,06015000203,2.03,Census Tract 2.03,G5020,S,368732963,35058418,+41.5767503,-124.0277960,POINT (-124.06094 41.55252)
4,12,1,DN,None,None,01C0020,143.808429,1.013075e+16,00000,41.553933,-124.061414,Salt Creek,0.1 Mi N/O Requa Rd,1975,5,Salt Creek Rd,4.0,15.9,64,0.1,0.1,3.8,N,0.0,1,3,1,Salt Creek,5,0,0,0,9,30-JAN-20,06,015,000203,06015000203,2.03,Census Tract 2.03,G5020,S,368732963,35058418,+41.5767503,-124.0277960,POINT (-124.06142 41.55393)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13327,13290,11,IMP,None,None,58C0150,-15.852287,1.125215e+16,00000,32.686011,-115.516239,All American (Kloke Rd),0.5 Mi N/O Of Sr-98,1965,5,Kloke Rd,9.1,40.2,394,0.0,0.0,9.8,N,0.0,5,2,1,All American Canal,1800,1,0,0,8,30-JAN-20,06,025,012201,06025012201,122.01,Census Tract 122.01,G5020,S,2169001,0,+32.6776062,-115.5099337,POINT (-115.51624 32.68601)
13328,13293,11,IMP,None,None,58C0161,152.391475,1.169845e+16,00000,32.997275,-115.543883,New River (Hovley Rd),1.2 Mi North Of St Rte 86,1989,5,Hovley Rd,8.2,42.1,430,0.0,0.0,9.8,N,0.0,4,2,1,New River,3500,2,0,0,8,30-JAN-20,06,025,010500,06025010500,105,Census Tract 105,G5020,S,3629627,0,+32.9875564,-115.5440020,POINT (-115.54389 32.99728)
13329,13323,11,IMP,None,None,58C0211,-104.983794,9.819280e+15,00000,32.861769,-115.594769,Central Main (Neckel Rd),50 Ft E/O Austin Rd,1972,5,Neckel Road,8.2,8.8,53,0.0,0.0,0.0,N,0.0,3,2,1,Central Main Canal,630,1,0,0,7,30-JAN-20,06,025,011002,06025011002,110.02,Census Tract 110.02,G5020,S,11530654,0,+32.8433039,-115.5795365,POINT (-115.59477 32.86177)
13330,13326,11,IMP,None,None,58C0215,-114.528673,9.956197e+15,00000,32.781367,-115.594789,Central Main (Ross Rd),50Ft E/O Austin Rd,1990,5,Ross Rd,7.3,9.1,234,0.0,0.0,0.0,N,0.0,3,2,1,Central Main Canal,467,1,0,0,16,30-JAN-20,06,025,011801,06025011801,118.01,Census Tract 118.01,G5020,S,3265947,0,+32.7832849,-115.5867423,POINT (-115.59479 32.78137)


In [47]:
bridge_data.columns

Index(['OBJECTID', 'DIST', 'CO', 'RTE', 'PM', 'BRIDGE', 'BRIDGE_X', 'BRIDGE_Y',
       'CITY', 'LAT', 'LON', 'NAME', 'LOC', 'YRBLT', 'HST', 'FAC', 'APWID',
       'LENG', 'DK_AREA', 'LSW', 'RSW', 'RDW', 'REFVCU', 'VCU', 'MAINSPANS',
       'DIR', 'PRINC', 'INTERSEC', 'AADT', 'PCTTRK', 'DEF', 'NHS',
       'FUNCTIONAL', 'DATA_EXTRA', 'USCB_STATEFP', 'USCB_COUNTYFP',
       'USCB_TRACTCE', 'USCB_GEOID', 'USCB_NAME', 'USCB_NAMELSAD',
       'USCB_MTFCC', 'USCB_FUNCSTAT', 'USCB_ALAND', 'USCB_AWATER',
       'USCB_INTPTLAT', 'USCB_INTPTLON', 'geometry'],
      dtype='object')

In [45]:
(pd.options.display.max_columns)
pd.options.display.max_columns = 500 # this will set limit of columns to 500

bridge_data


,OBJECTID,DIST,CO,RTE,PM,BRIDGE,BRIDGE_X,BRIDGE_Y,CITY,LAT,LON,NAME,LOC,YRBLT,HST,FAC,APWID,LENG,DK_AREA,LSW,RSW,RDW,REFVCU,VCU,MAINSPANS,DIR,PRINC,INTERSEC,AADT,PCTTRK,DEF,NHS,FUNCTIONAL,DATA_EXTRA,USCB_STATEFP,USCB_COUNTYFP,USCB_TRACTCE,USCB_GEOID,USCB_NAME,USCB_NAMELSAD,USCB_MTFCC,USCB_FUNCSTAT,USCB_ALAND,USCB_AWATER,USCB_INTPTLAT,USCB_INTPTLON,geometry
0,1,1,DN,None,None,01C0001,-0.862156,9.486604e+15,00000,41.523222,-124.032594,Hoppow Creek,500' E Of Rt 101/169 Sep,1965,5,Klamath Mill Road,8.5,17.7,200,1.5,0.6,8.5,N,0.0,2,2,1,Hoppow Creek,1395,25,0,0,9,30-JAN-20,06,015,000203,06015000203,2.03,Census Tract 2.03,G5020,S,368732963,35058418,+41.5767503,-124.0277960,POINT (-124.03260 41.52322)
1,2,1,DN,None,None,01C0002,-54.146810,9.519316e+15,00000,41.523681,-124.035647,Hoppow Creek,West Of Route 101/169 Sep,1965,5,Chapman Street,9.4,31.0,381,0.6,1.5,8.5,N,0.0,1,2,1,Hoppow Creek,20,0,0,0,9,30-JAN-20,06,015,000203,06015000203,2.03,Census Tract 2.03,G5020,S,368732963,35058418,+41.5767503,-124.0277960,POINT (-124.03565 41.52368)
2,7,1,DN,None,None,01C0011,-42.523364,9.819594e+15,00000,41.553397,-124.057572,Hunter Creek,0.1 Mi West Of Rt101,1949,5,Requa Road,8.4,41.6,356,0.0,0.0,8.4,N,0.0,1,2,1,Hunter Creek,1110,2,0,0,8,30-JAN-20,06,015,000203,06015000203,2.03,Census Tract 2.03,G5020,S,368732963,35058418,+41.5767503,-124.0277960,POINT (-124.05757 41.55339)
3,8,1,DN,None,None,01C0012,37.575706,9.853622e+15,00000,41.552528,-124.060939,Salt Creek,0.4 Mi West Of Rte 101,1958,5,Requa Road,8.2,7.0,73,0.0,0.0,0.0,N,0.0,2,2,1,Salt Creek,1110,2,0,0,8,30-JAN-20,06,015,000203,06015000203,2.03,Census Tract 2.03,G5020,S,368732963,35058418,+41.5767503,-124.0277960,POINT (-124.06094 41.55252)
4,12,1,DN,None,None,01C0020,143.808429,1.013075e+16,00000,41.553933,-124.061414,Salt Creek,0.1 Mi N/O Requa Rd,1975,5,Salt Creek Rd,4.0,15.9,64,0.1,0.1,3.8,N,0.0,1,3,1,Salt Creek,5,0,0,0,9,30-JAN-20,06,015,000203,06015000203,2.03,Census Tract 2.03,G5020,S,368732963,35058418,+41.5767503,-124.0277960,POINT (-124.06142 41.55393)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13327,13290,11,IMP,None,None,58C0150,-15.852287,1.125215e+16,00000,32.686011,-115.516239,All American (Kloke Rd),0.5 Mi N/O Of Sr-98,1965,5,Kloke Rd,9.1,40.2,394,0.0,0.0,9.8,N,0.0,5,2,1,All American Canal,1800,1,0,0,8,30-JAN-20,06,025,012201,06025012201,122.01,Census Tract 122.01,G5020,S,2169001,0,+32.6776062,-115.5099337,POINT (-115.51624 32.68601)
13328,13293,11,IMP,None,None,58C0161,152.391475,1.169845e+16,00000,32.997275,-115.543883,New River (Hovley Rd),1.2 Mi North Of St Rte 86,1989,5,Hovley Rd,8.2,42.1,430,0.0,0.0,9.8,N,0.0,4,2,1,New River,3500,2,0,0,8,30-JAN-20,06,025,010500,06025010500,105,Census Tract 105,G5020,S,3629627,0,+32.9875564,-115.5440020,POINT (-115.54389 32.99728)
13329,13323,11,IMP,None,None,58C0211,-104.983794,9.819280e+15,00000,32.861769,-115.594769,Central Main (Neckel Rd),50 Ft E/O Austin Rd,1972,5,Neckel Road,8.2,8.8,53,0.0,0.0,0.0,N,0.0,3,2,1,Central Main Canal,630,1,0,0,7,30-JAN-20,06,025,011002,06025011002,110.02,Census Tract 110.02,G5020,S,11530654,0,+32.8433039,-115.5795365,POINT (-115.59477 32.86177)
13330,13326,11,IMP,None,None,58C0215,-114.528673,9.956197e+15,00000,32.781367,-115.594789,Central Main (Ross Rd),50Ft E/O Austin Rd,1990,5,Ross Rd,7.3,9.1,234,0.0,0.0,0.0,N,0.0,3,2,1,Central Main Canal,467,1,0,0,16,30-JAN-20,06,025,011801,06025011801,118.01,Census Tract 118.01,G5020,S,3265947,0,+32.7832849,-115.5867423,POINT (-115.59479 32.78137)


In [34]:
print('length of airport data before county filter:', len(airport_data))
airport_counties, omitted_airports = filter_counties(airport_data, 'COUNTY', county_list=None)
print('length of aiprort data after county filter:', len(airport_counties))
print('')

print('Omitted rows:')
omitted_airports = omitted_airports['COUNTY']
display(omitted_airports)
print('')

print('fixing spelling issue so airport data is ready for future steps:')
cleaned_airport_data = airport_data
cleaned_airport_data['COUNTY'] = airport_counties["COUNTY"].replace('Humbolt', 'Humboldt')
print('length of cleaned airport data:', len(cleaned_airport_data))

length of airport data before county filter: 242
length of aiprort data after county filter: 241

Omitted rows:


189    NaN
Name: COUNTY, dtype: object


fixing spelling issue so airport data is ready for future steps:
length of cleaned airport data: 242


In [35]:
print('length of airport data before county filter:', len(bridge_data))
bridge_counties, omitted_bridge = filter_counties(bridge_data, 'COUNTY', county_list=None)
print('length of aiprort data after county filter:', len(bridge_counties))
print('')

print('Omitted rows:')
omitted_bridge = omitted_bridge['COUNTY']
display(omitted_bridge)
print('')
'''
print('fixing spelling issue so airport data is ready for future steps:')
cleaned_airport_data = airport_data
cleaned_airport_data['COUNTY'] = airport_counties["COUNTY"].replace('Humbolt', 'Humboldt')
print('length of cleaned airport data:', len(cleaned_airport_data))'''

length of airport data before county filter: 242
length of aiprort data after county filter: 241

Omitted rows:


189    Humbolt
Name: COUNTY, dtype: object

'\nprint(\'fixing spelling issue so airport data is ready for future steps:\')\ncleaned_airport_data = airport_data\ncleaned_airport_data[\'COUNTY\'] = airport_counties["COUNTY"].replace(\'Humbolt\', \'Humboldt\')\nprint(\'length of cleaned airport data:\', len(cleaned_airport_data))'

In [4]:
# read in CA census tiger file
census_shp_dir = "s3://ca-climate-index/0_map_data/2021_tiger_census_tract/2021_ca_tract/"
ca_boundaries = gpd.read_file(census_shp_dir)

# Create a copy of the relevant columns from ca_boundaries
filtered_ca_boundaries = ca_boundaries[['GEOID', 'geometry']].copy()

# Rename the 'GEOID' column to 'tract'
filtered_ca_boundaries.rename(columns={'GEOID': 'tract'}, inplace=True)

# Remove the first character from the 'tract' column
filtered_ca_boundaries['tract'] = filtered_ca_boundaries['tract'].str[1:]

# Display the resulting DataFrame
filtered_ca_boundaries

,tract,geometry
0,6085504321,"POLYGON ((-121.87556 37.39924, -121.87535 37.3..."
1,6085504410,"POLYGON ((-121.88886 37.40758, -121.88576 37.4..."
2,6085507003,"POLYGON ((-122.02489 37.21683, -122.02459 37.2..."
3,6085507004,"POLYGON ((-121.99304 37.22562, -121.99249 37.2..."
4,6085502204,"POLYGON ((-121.93167 37.29803, -121.92801 37.3..."
...,...,...
9124,6059001303,"POLYGON ((-117.95917 33.92458, -117.95888 33.9..."
9125,6059001304,"POLYGON ((-117.95918 33.92820, -117.95831 33.9..."
9126,6059001401,"POLYGON ((-117.95056 33.94503, -117.95055 33.9..."
9127,6013367200,"POLYGON ((-122.34551 37.96355, -122.34550 37.9..."


In [77]:
# read in CA census tiger file
ca_tract_county = "s3://ca-climate-index/0_map_data/ca_tracts_county.csv"
ca_tract_county = gpd.read_file(ca_tract_county)
ca_tract_county = ca_tract_county.drop(columns={'field_1', 'geometry'})
ca_tract_county.columns = ca_tract_county.columns.str.lower()
ca_tract_county = ca_tract_county.applymap(lambda s: s.lower() if type(s) == str else s)

ca_tract_county

,tract,countyfp,county
0,06085504321,085,santa clara
1,06085504410,085,santa clara
2,06085507003,085,santa clara
3,06085507004,085,santa clara
4,06085502204,085,santa clara
...,...,...,...
9124,06059001303,059,orange
9125,06059001304,059,orange
9126,06059001401,059,orange
9127,06013367200,013,contra costa


In [78]:
transportation_infrastructure_data = [airport_data, bridge_data]
words_to_search = ['AIRPORTID','COUNTY', 'USCB_COUNTYFP', 'BRIDGE', 'geometry']

def filter_and_spatial_join(data_list, filtered_ca_boundaries, words_to_search, ca_tract_county):
    county_count_dfs = {}
    
    # Print the columns of ca_tract_county to understand its structure
    print(f"Columns in ca_tract_county: {ca_tract_county.columns.tolist()}")
    
    for df, df_name in zip(data_list, ['airport_data', 'bridge_data']):        
        # Filter columns based on words_to_search
        filtered_df = df[[col for col in df.columns if any(word in col for word in words_to_search)]].copy()
        filtered_df.index = df.index
        filtered_df = filtered_df.to_crs(crs=4269)

        # Convert all string columns to lowercase
        str_columns = filtered_df.select_dtypes(include=['object']).columns
        for col in str_columns:
            filtered_df[col] = filtered_df[col].str.lower()

        # Perform the spatial join
        joined_df = gpd.sjoin(filtered_df, filtered_ca_boundaries, how='right', predicate='within')
        
        # Ensure necessary columns are retained
        necessary_columns = ['AIRPORTID', 'COUNTY', 'USCB_COUNTYFP', 'BRIDGE', 'geometry']
        joined_df = joined_df[[col for col in necessary_columns if col in joined_df.columns]].copy()
        
        # Debug statement to check joined_df columns
        print(f"Joined DataFrame columns for {df_name}: {joined_df.columns.tolist()}")
                
        if 'AIRPORTID' in joined_df.columns:
            id_column = 'AIRPORTID'
        elif 'BRIDGE' in joined_df.columns:
            id_column = 'BRIDGE'
        else:
            raise ValueError(f"Neither 'AIRPORTID' nor 'BRIDGE' found in the DataFrame for {df_name}")
        
        # Determine county_id
        if 'COUNTY' in joined_df.columns:
            joined_df = joined_df.rename(columns={'COUNTY': 'county'})
            county_id = 'county'
        elif 'USCB_COUNTYFP' in joined_df.columns:
            joined_df = joined_df.rename(columns={'USCB_COUNTYFP': 'countyfp'})
            county_id = 'countyfp'
        else:
            raise ValueError(f"Neither 'COUNTY' nor 'USCB_COUNTYFP' found in the DataFrame for {df_name}")

        # Debug statement to check for county_id column
        print(f"Using county_id: {county_id} for {df_name}")
        
        # Remove duplicates based on county and the chosen ID column
        unique_communication_structures_county = joined_df.drop_duplicates(subset=[county_id, id_column])

        county_power_counts = unique_communication_structures_county.groupby(county_id)[id_column].apply(lambda x: x.notnull().sum()).reset_index(name=f"{df_name}_count")
        
        # Debug statement to check the counts
        print(f"County power counts for {df_name}:")
        print(county_power_counts.head())

        # Normalize column names for merging
        ca_tract_county.columns = ca_tract_county.columns.str.lower()
        county_power_counts.columns = county_power_counts.columns.str.lower()

        # Check which column to use for merging
        merge_col = county_id  # use the county_id determined above
        
        # Merge with ca_tract_county
        merged_df = pd.merge(ca_tract_county, county_power_counts, on=merge_col, how='left')
        
        county_df_name = f"county_count_{df_name}"
        
        county_count_dfs[county_df_name] = merged_df
        
        # Dynamically create global variables
        globals()[county_df_name] = merged_df
        
        print(county_df_name)
    
    return county_count_dfs

county_count_dfs = filter_and_spatial_join(transportation_infrastructure_data, filtered_ca_boundaries, words_to_search, ca_tract_county)

Columns in ca_tract_county: ['tract', 'countyfp', 'county']
Joined DataFrame columns for airport_data: ['AIRPORTID', 'COUNTY', 'USCB_COUNTYFP', 'geometry']
Using county_id: county for airport_data
County power counts for airport_data:
      county  airport_data_count
0    alameda                   3
1     alpine                   1
2     amador                   1
3      butte                   2
4  calaveras                   1
county_count_airport_data
Joined DataFrame columns for bridge_data: ['USCB_COUNTYFP', 'BRIDGE', 'geometry']
Using county_id: countyfp for bridge_data
County power counts for bridge_data:
  countyfp  bridge_data_count
0      001                302
1      003                 12
2      005                 42
3      007                306
4      009                 69
county_count_bridge_data


In [57]:
county_count_bridge_data

,tract,countyfp,county,bridge_data_count
0,06085504321,085,Santa Clara,508
1,06085504410,085,Santa Clara,508
2,06085507003,085,Santa Clara,508
3,06085507004,085,Santa Clara,508
4,06085502204,085,Santa Clara,508
...,...,...,...,...
9124,06059001303,059,Orange,586
9125,06059001304,059,Orange,586
9126,06059001401,059,Orange,586
9127,06013367200,013,Contra Costa,362


In [80]:
def county_count(df, county_col, county, counter):
    county_isolate = df[df[county_col]==county]
    county_isolate_drop_duplicates= county_isolate.drop_duplicates(subset=[county_col, counter])
    print(f'Length of df for {county} county without dropping duplicates:  {len(county_isolate)}')
    print(f'Length of df for {county} county after dropping duplicates: {len(county_isolate_drop_duplicates)}')

county_count(airport_data, 'COUNTY', 'Orange', 'AIRPORTID')

Length of df for Orange county without dropping duplicates:  2
Length of df for Orange county after dropping duplicates: 2


In [79]:
county_count_airport_data

,tract,countyfp,county,airport_data_count
0,06085504321,085,santa clara,4.0
1,06085504410,085,santa clara,4.0
2,06085507003,085,santa clara,4.0
3,06085507004,085,santa clara,4.0
4,06085502204,085,santa clara,4.0
...,...,...,...,...
9124,06059001303,059,orange,2.0
9125,06059001304,059,orange,2.0
9126,06059001401,059,orange,2.0
9127,06013367200,013,contra costa,2.0


In [ ]:
# Filter the DataFrame to include only rows where 'CECPlantID' column exists
checkin = cleaned_power_plants.dropna(subset=['CECPlantID'])
county_count = checkin['County'] == 'Kern'
print('Number of entries of designated county:',len(county_count[county_count]))